# MMR To MF6 DFW

Perform MF6 diffussive wave routing (DFW) simulaton based on PRMS MMR (Muskingum-Mann Routing) inputs and boundary flows from the PRMS model. 

This notebook requries the develop branch of MF6 and a flopy upto date with this branch by running `python update_flopy.py` in modflow6/autotest. The notebook also requires running the first notebook in examples, `00_processes.ipynb` to obtain the shapefiles for the domain.

## User configuration

In [ ]:
# Set YOUR path to MF6 here
import pathlib as pl

mf6_bin = pl.Path("../../modflow6/bin/mf6")
assert (
    mf6_bin.exists
), "A build of mf6/develop branch required for DFW simulation"

# Rerun MF6 model below or use an existing run?
rerun_mf6 = True

## Preparation

In [ ]:
import os
import shutil

import contextily as cx
import flopy
import geopandas as gpd
import jupyter_black
import numpy as np
import pandas as pd
import pint
import pywatershed as pws
import xarray as xr

import hvplot.xarray  # noqa

import pywatershed as pws
from pywatershed.utils.mmr_to_mf6_dfw import MmrToMf6Dfw

jupyter_black.load()

repo_root_dir = pws.constants.__pywatershed_root__.parent
test_data_dir = repo_root_dir / "test_data"

os.environ["PATH"] += os.pathsep + str(mf6_bin.parent)

plot_height = 350
plot_width = 900

## Setup and run MF6 DFW on the Delaware river basin using PRMS parameters and influxes

In [ ]:
domain = "drb_2yr"
domain_dir = test_data_dir / f"{domain}"
run_dir = repo_root_dir / f"examples/tmp_{domain}_mmr_to_mf6_dfw"

This is where the inflows from PRMS live in PRMS output files:

In [ ]:
inflow_dir = domain_dir / "output"

In [ ]:
control_file = domain_dir / "nhm.control"
control = pws.Control.load_prms(control_file)
ndays_run = control.n_times
# Could shorten the run duration
# Subtract one from ndays_run becase end day/time would be included in the PRMS run
# control.edit_end_time(
#     control.start_time + ((ndays_run-1) * control.time_step)
# )

In [ ]:
dis_both_file = domain_dir / "parameters_dis_both.nc"
dis_both = pws.Parameters.from_netcdf(dis_both_file)
dis_both_ds = xr.open_dataset(dis_both_file)
seg_params = pws.Parameters.from_netcdf(
    domain_dir / "parameters_PRMSChannel.nc"
)
params = pws.Parameters.merge(dis_both, seg_params)

seg_shp_file = (
    repo_root_dir
    / "pywatershed/data/pywatershed_gis/drb_2yr/Segments_subset.shp"
)

In [ ]:
# IMS options
nouter, ninner = 100, 50
hclose, rclose, relax = 1e-5, 1.0, 0.97

ims_options = {
    "print_option": "SUMMARY",
    "outer_dvclose": hclose,
    "outer_maximum": nouter,
    "under_relaxation": "DBD",
    "under_relaxation_theta": 0.95,
    "under_relaxation_kappa": 0.0001,
    "under_relaxation_gamma": 0.0,
    "under_relaxation_momentum": 0.0,
    "inner_maximum": ninner,
    "inner_dvclose": hclose,
    "linear_acceleration": "BICGSTAB",
    "scaling_method": "NONE",
    "reordering_method": "NONE",
    "relaxation_factor": relax,
    "filename": "drb.ims",
}

dfw_options = {
    "print_flows": True,
    "save_flows": True,
    "idcxs": None,  # zero based in flopy, None for hydraulically wide
}

sto_options = {"save_flows": True}

oc_options = {
    "saverecord": [
        ("STAGE", "ALL"),
        ("BUDGET", "ALL"),
    ],
    "printrecord": [
        ("STAGE", "LAST"),
        ("BUDGET", "ALL"),
    ],
}

# Initial water depth, units of meters
ic_options = {"strt": 0.5}

chd_options = {
    "print_input": True,
    "print_flows": False,
}

bc_binary_files = True
bc_flows_combined = True

In [ ]:
%%time
# This takes about 11 minutes on my Mac

tdis_perlen = 2 * 3600  # stress period length
tdis_nstp = 3  # substeps per stress period

if rerun_mf6:
    if run_dir.exists():
        shutil.rmtree(run_dir)
    run_dir.mkdir(parents=True)

    dfw = MmrToMf6Dfw(
        control=control,
        segment_shp_file=seg_shp_file,
        params=params,
        tdis_perlen=tdis_perlen,
        tdis_nstp=tdis_nstp,
        output_dir=run_dir,
        sim_name="drb_dfw",
        ims_options=ims_options,
        dfw_options=dfw_options,
        sto_options=sto_options,
        oc_options=oc_options,
        ic_options=ic_options,
        chd_options=chd_options,
        bc_binary_files=bc_binary_files,
        bc_flows_combined=bc_flows_combined,
        inflow_dir=inflow_dir,
    )

    success, buff = dfw.run(silent=False, report=True)
    assert success

## Get DFW stage and flow for plotting

In [ ]:
sim = flopy.mf6.MFSimulation.load(
    "drb_mf6_dfw",
    sim_ws=str(run_dir),
    exe_name="mf6",
    load_only=["disv1d"],
)
sim.model_names
model = sim.get_model("drb_dfw")

In [ ]:
# time
tdis = sim.tdis
sim_start_time = np.datetime64(tdis.start_date_time.get_data().upper()[0:19])
n_substeps = int(ndays_run * 24 * 60 * 60 / tdis_perlen * tdis_nstp)
substep_len = np.timedelta64(int(tdis_perlen / tdis_nstp), "s")
sim_end_time = sim_start_time + n_substeps * substep_len
sim_times = np.arange(sim_start_time, sim_end_time, substep_len)
perioddata = tdis.perioddata.get_data()
assert len(sim_times) == len(perioddata) * perioddata[0][1]

In [ ]:
# stage
stage_file = run_dir / "drb_dfw.stage"
sobj = flopy.utils.HeadFile(stage_file, text="STAGE", verbose=False)
stage_all = sobj.get_alldata().squeeze()
disv1d = model.get_package("disv1d")
bottom_ele = disv1d.bottom.get_data()
stage_all = np.maximum(stage_all - bottom_ele, 0)

In [ ]:
# getting flow is more complicated and could be improved/refined
budget_file = run_dir / "drb_dfw.bud"
budobj = flopy.utils.binaryfile.CellBudgetFile(budget_file)
flowja = budobj.get_data(text="FLOW-JA-FACE")
# qstorage = budobj.get_data(text="STORAGE")
# qflw = budobj.get_data(text="FLW")
# qextoutflow = budobj.get_data(text="EXT-OUTFLOW")

grb_file = run_dir / "drb_dfw.disv1d.grb"
grb = flopy.mf6.utils.MfGrdFile(grb_file)
ia = grb.ia
ja = grb.ja


def get_outflow(itime):
    outflow = np.zeros(ia.shape[0] - 1)
    flowjaflat = flowja[itime].flatten()
    for n in range(grb.nodes):
        for ipos in range(ia[n] + 1, ia[n + 1]):
            q = flowjaflat[ipos]
            if q < 0:
                outflow[n] += -q
    # <<<
    return outflow


flow_all = stage_all.copy() * np.nan
for tt in range(flow_all.shape[0]):
    flow_all[tt, :] = get_outflow(tt)

In [ ]:
# Create an xarray dataset with MF6 DFW stage and flow
mf6_ds = (
    xr.DataArray(
        flow_all,
        dims=["time", "nhm_seg"],
        coords={"time": sim_times, "nhm_seg": dis_both_ds.nhm_seg.values},
    )
    .rename("dfw_flow")
    .to_dataset()
)
mf6_ds["dfw_stage"] = xr.DataArray(
    stage_all * 100,
    dims=["time", "nhm_seg"],
    coords={"time": sim_times, "nhm_seg": dis_both_ds.nhm_seg.values},
)

In [ ]:
# SAVE FOR LATER?
# dis_ds = xr.open_dataset(dis_both_file)
# time_ind = 16055
# flowjaflat = flowja[time_ind].flatten()

# ind = 170

# slice_ind = slice(ia[ind], ia[ind + 1])
# fja_ind = flowjaflat[slice_ind]
# print(f"{ind=}")
# print(f"{slice_ind=}")
# print(f"{ja[slice_ind]=}")
# print(f"{flowjaflat[slice_ind]=}")

# for other_ind in ja[slice_ind][1:]:
#     print(f"\n{other_ind=}")
#     if dis_ds.tosegment[ind] - 1 == other_ind:
#         print("reach below")
#     elif dis_ds.tosegment[other_ind] - 1 == ind:
#         print("reach above")
#     else:
#         print("reach also flowing to below reach")
#         ind_below = dis_ds.tosegment[ind] - 1
#         inds_above_below = np.where(dis_ds.tosegment == (ind_below + 1))
#         assert other_ind in inds_above_below[0]

#     slice_other = slice(ia[other_ind], ia[other_ind + 1])
#     print(f"{slice_other=}")
#     print(f"{ja[slice_other]=}")

#     fja_other = flowjaflat[slice_other]
#     print(f"{flowjaflat[slice_other]=}")

## Make a spatial plot of flow

In [ ]:
shp_df = gpd.read_file(seg_shp_file).sort_values(
    "model_idx", axis=0, ignore_index=True
)

In [ ]:
flow_log = np.maximum(np.log(flow_all + 1.0e-4), 1)

width_max = 6
width_min = 0.2
flow_log_lw = (width_max - 1) * (flow_log - np.min(flow_log, axis=0)) / (
    np.max(flow_log, axis=0) - np.min(flow_log, axis=0)
) + width_min

In [ ]:
tt = 3000
zoom = 1.0
figsize = (7 * zoom, 10 * zoom)
dt = tdis_perlen / tdis_nstp
current_time = sim_times[tt]
ax = shp_df.plot(
    column="model_idx",
    figsize=figsize,
    linewidth=flow_log_lw[tt, :],
    edgecolor="darkblue",
)

cx.add_basemap(ax=ax, crs=shp_df.crs, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()
_ = ax.set_title(f"{current_time}")

## Compare PRMS MMR and MF6 DFW solutions with observations

In [ ]:
# bing in PRMS flows and convert units
prms_flow_da = xr.open_dataarray(domain_dir / "output/seg_outflow.nc")
prms_flow_da = prms_flow_da.reset_index("time")

# The PRMS flows are "daily" so we'll center them on the day then interpolate
# to the the same times as MF6 DFW
prms_flow_da["time"] = prms_flow_da["time"] + np.timedelta64(12, "h")
prms_flow_da = prms_flow_da.where(prms_flow_da.time.isin(sim_times), drop=True)
flow_ds = (
    prms_flow_da.resample(time=f"{int(tdis_perlen/tdis_nstp)}s")
    .interpolate("nearest")
    .to_dataset(name="prms")
)

# Convert units using pint
units = pint.UnitRegistry()
flow_ds["prms"][:, :] = (
    (flow_ds["prms"].values * units("feet ** 3 / second")).to(
        "meters ** 3 / second"
    )
).m

In [ ]:
# This dataset is at 40 minute resolution
prms_mf6_ds = xr.merge([mf6_ds, flow_ds])[
    ["prms", "dfw_flow", "dfw_stage"]
].rename(
    {
        "prms": "prms streamflow",
        "dfw_flow": "dfw streamflow",
        "dfw_stage": "dfw stage",
    }
)

In [ ]:
# Subset to points of interest (poi), known flow gages
poi_id = np.chararray(
    prms_mf6_ds["prms streamflow"].nhm_seg.shape, unicode=True, itemsize=15
)
empty_str = " " * 15
poi_id[:] = empty_str
for ii, jj in enumerate(params.parameters["poi_gage_segment"].tolist()):
    poi_id[jj] = params.parameters["poi_gage_id"][ii]

prms_mf6_ds["poi_id"] = xr.DataArray(poi_id, dims=["nhm_seg"])

prms_mf6_poi_ds = (
    prms_mf6_ds[["prms streamflow", "dfw streamflow", "poi_id"]]
    .where(prms_mf6_ds.poi_id != empty_str, drop=True)
    .set_coords("poi_id")
    .swap_dims(nhm_seg="poi_id")
)

In [ ]:
observations_nc_file = domain_dir / "drb_2yr_gage_poi_obs.nc"
obs_da = xr.open_dataset(observations_nc_file)["discharge"].rename("observed")
obs_da[:] = (
    (obs_da.values * units("feet ** 3 / second"))
    .to("meters ** 3 / second")
    .magnitude
)
obs_ds = obs_da.to_dataset()
obs_all_na = np.isnan(obs_ds.observed).sum(dim="time") == len(obs_ds.time)
obs_ds = obs_ds.where(~obs_all_na, drop=True)
# make it 40 minute resolution
obs_ds = obs_ds.resample(time=f"{int(tdis_perlen/tdis_nstp)}s").interpolate(
    "nearest"
)

In [ ]:
eval_ds = xr.merge([obs_ds, prms_mf6_poi_ds], join="inner").set_xindex(
    "nhm_seg"
)
eval_ds

There were 103 gages with observations at some times. Let's take a look at the first three.

In [ ]:
eval_plot_ds = eval_ds[
    [
        "observed",
        "dfw streamflow",
        "prms streamflow",
    ]
]


def plot_eval_ds(poi_id):
    display(
        eval_plot_ds.sel(poi_id=poi_id).hvplot(
            x="time",
            # groupby="poi_id",  # not working for me currently, just looping over some instead
            ylabel="streamflow (m^3/s)",
            xlabel="",
            height=350,
            width=800,
        )
    )


for ii in eval_ds.poi_id[0:3]:
    plot_eval_ds(ii)

## Trenton Gage
Which observered point has the largest mean flow?

In [ ]:
wh_max_flow = eval_ds.observed.mean(dim="time").argmax(dim="poi_id")
poi_id_max_flow = str(eval_ds.poi_id[wh_max_flow].values[()])

In [ ]:
plot_eval_ds(poi_id_max_flow)

Hold on. Someting looks really wrong at this gage. PRMS and MF6 dont match and PRMS is much lower than the observations. What is going on?

What we'll find is that the gage has been registered to the wrong segment in PRMS. We'll also see that DFW models this more close to the observations because this "wrong" reach is hydraulically connected in DFW to where the gage is actually located.

Where is this going on?

In [ ]:
import dataretrieval.nwis as nwis

trenton_meta = nwis.get_record(sites=poi_id_max_flow, service="site")
print(trenton_meta["station_nm"][0])

In [ ]:
# # do this after the FlowGraph branch merges
# plot = pws.plot.DomainPlot(
#     hru_shp_file=shp_file_hru,
#     segment_shp_file=seg_shp_file,
#     hru_parameters=domain_dir / "parameters_dis_hru.nc",
#     hru_parameter_names=[
#         "hru_lat",
#         "hru_lon",
#         "hru_area",
#     ],
#     seg_parameters=domain_dir / "parameters_dis_seg.nc",
#     seg_param_names = [
#         "nhm_seg",
#         "seg_cum_area",
#         "seg_length",
#         "seg_slope",
#         "tosegment_nhm",
#     ],
#     start_lat=trenton_meta.dec_lat_va[0],
#     start_lon=trenton_meta.dec_long_va[0],
#     start_zoom=12,
#     display_plot=False,
# )

In [ ]:
def plot_modeled_location(
    nhm_seg,
    logy=False,
    plot_vars=["prms streamflow", "dfw streamflow"],
):

    ylabel = ""
    any_flow = [True for ss in plot_vars if "streamflow" in ss]
    if len(any_flow):
        ylabel += "streamflow (m^3/s)"
    if "dfw stage" in plot_vars:
        if len(any_flow):
            ylabel += "\n"
        ylabel += "stage (cm)"

    display(
        prms_mf6_ds[plot_vars]
        .sel(nhm_seg=nhm_seg)
        .hvplot(
            x="time",
            ylabel=ylabel,
            xlabel="",
            height=plot_height,
            width=plot_width,
            logy=logy,
        )
    )

In [ ]:
plot_modeled_location(nhm_seg=1498)
plot_modeled_location(nhm_seg=1493)
plot_modeled_location(nhm_seg=1499)
plot_modeled_location(nhm_seg=1766)

Put the observed flows on the seemingly more appropriate, upstream segment with `nhm_seg=1498`.

In [ ]:
seg_1498_ds = prms_mf6_ds.where(prms_mf6_ds.nhm_seg.isin([1498]), drop=True)
seg_1498_ds["poi_id"] = xr.DataArray(
    np.array([poi_id_max_flow]), dims=["nhm_seg"]
)
seg_1498_ds = seg_1498_ds.swap_dims(nhm_seg="poi_id")
trenton_gage_eval_ds = xr.merge([obs_ds, seg_1498_ds], join="inner")

In [ ]:
display(
    trenton_gage_eval_ds[["observed", "dfw streamflow", "prms streamflow"]]
    .sel(poi_id=poi_id_max_flow)
    .hvplot(
        x="time",
        ylabel="streamflow (m^3/s)",
        xlabel="",
        height=plot_height,
        width=plot_width,
    )
)

Let's consider the river stages modeled by DFW in the 4 links including and connecting to the one with `nhm_seg=1499` where the gage was erroneously placed, to verify that they have very similar heads as this is what makes DFW more realistic on this wrong reach. 

In [ ]:
plot_modeled_location(nhm_seg=1498, plot_vars=["dfw stage"])
plot_modeled_location(nhm_seg=1493, plot_vars=["dfw stage"])
plot_modeled_location(nhm_seg=1499, plot_vars=["dfw stage"])
plot_modeled_location(nhm_seg=1766, plot_vars=["dfw stage"])